In [1]:
import syft as sy
from syft.store.blob_storage import BlobStorageConfig, BlobStorageClientConfig
from syft.store.blob_storage.seaweedfs import SeaweedFSClient, SeaweedFSClientConfig
from syft import ActionObject
from syft.service.action.action_data_empty import ActionFileData
from syft.service.queue.zmq_queue import ZMQQueueConfig, ZMQClientConfig
from collections import defaultdict

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node = sy.orchestra.launch(name="test-domain-helm2", dev_mode=True,
                           reset=True,
                           n_consumers=4,
                           create_producer=True)
client = node.login(email="info@openmined.org", password="changethis")

INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
INITIALIZING CONSUMER
ABCDEF
Logged into <test-domain-helm2: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

```bash
docker run --entrypoint /bin/sh -p 8333:8333 -p 8888:8888 chrislusf/seaweedfs -c "echo 's3.configure -access_key admin -secret_key admin -user iam -actions Read,Write,List,Tagging,Admin -apply' | weed shell > /dev/null 2>&1 & weed server -s3 -s3.port=8333 -master.volumeSizeLimitMB=2048"
```

In [3]:
blob_config = BlobStorageConfig(client_type=SeaweedFSClient,
                                client_config=SeaweedFSClientConfig(host="http://0.0.0.0",
                                                                    port="8333",
                                                                    access_key="admin",
                                                                    secret_key="admin",
                                                                    bucket_name="test_bucket",
                                                                    region="us-east-1")
)

In [4]:
node.python_node.init_blob_storage(blob_config)

# Inputs

In [5]:
# ActionObject.from_obj([sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data] * 2 )

In [6]:
sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data

```python
class BlobFile:
  id: str = 5da9b91f944346fdb1744929acb3efa2

```

In [7]:
# TODO: fix way we send list of files
scenario_obj = ActionObject.from_obj([
    sy.ActionObject.from_path(path="scenario_data.jsonl").send(client).syft_action_data for i in range(2)])
scenario_files_ptr = scenario_obj.send(client)

In [8]:
# TODO: fix way we send list of files
input_obj = ActionObject.from_obj([
    sy.ActionObject.from_path("short_input.jsonl").send(client).syft_action_data for i in range(2)])
input_files_ptr = input_obj.send(client)

In [9]:
# for line in input_files_ptr.syft_action_data[0].iter_lines():
#     print(line)

# Syft functions

In [10]:
@sy.syft_function()
def compute_document_data_overlap(scenario_file, input_files, n):
    print("starting overlap computation")

    from nltk import ngrams
    from collections import defaultdict
    from string import punctuation
    import re, json

    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    
    def create_ngram_index(light_scenarios, n_values, stats_key_counts):
        ngram_index = {n:{}  for n in n_values}
        for scenario in light_scenarios:
            for n in n_values:
                stats_key = scenario['scenario_key'] + '_' + str(n)
                stats_key_counts[stats_key] = len(scenario['instances'])
                for instance in scenario['instances']:
                    id = instance['id']                    
                    input_tokens = r.split(instance['input'].lower())
                    for input_ngram in ngrams(input_tokens, n):
                        if input_ngram not in ngram_index[n]:
                            ngram_index[n][input_ngram] = set()
                        ngram_index[n][input_ngram].add(stats_key + '+' + id + '+' + 'input')

                    # compute reference ngrams
                    for reference in instance['references']:
                        reference_unigrams = r.split(reference.lower())
                        for reference_ngram in ngrams(reference_unigrams, n):
                            if reference_ngram not in ngram_index[n]:
                                ngram_index[n][reference_ngram] = set()
                            ngram_index[n][reference_ngram].add(stats_key + '+' + id + '+' + 'references')
        return ngram_index
    
    # # SETUP
    print("preparing scenarios and creating indexes")
    light_scenarios = []
    for light_scenario_json in scenario_file.iter_lines():
        light_scenario_dict: dict = json.loads(light_scenario_json)

        light_scenario_key_dict: dict = light_scenario_dict["scenario_key"]
        subject_spec = light_scenario_key_dict["scenario_spec"]['args']['subject']
        light_scenario_key = subject_spec + '_' + light_scenario_key_dict["split"]
        light_instances = [
            {
                'input': instance_dict['input'], 
                'references': instance_dict['references'], 
                'id': instance_dict["id"]
            }
            for instance_dict in light_scenario_dict["instances"]
        ]
        light_scenarios.append({'scenario_key': light_scenario_key, 'instances': light_instances})
        
    stats_key_counts = defaultdict(int)
    
    ngram_index = create_ngram_index(
        light_scenarios=light_scenarios, n_values=[n], stats_key_counts=stats_key_counts
    )
    
    r = re.compile(r"[\s{}]+".format(re.escape(punctuation)))
    stats_key_to_input_ids = defaultdict(set)
    stats_key_to_reference_ids = defaultdict(set)
    print("computing overlap")
    
    for input_file in input_files:
        for line in input_file.iter_lines():
            document = json.loads(line)["text"]
            document_tokens = r.split(document.lower())
            for n in ngram_index.keys():
                for document_ngram in ngrams(document_tokens, n):
                    if document_ngram in ngram_index[n]:
                        for entry_overlap_key in ngram_index[n][document_ngram]:
                            stats_key, id, part = entry_overlap_key.split("+")
                            if part == "input":
                                stats_key_to_input_ids[stats_key].add(id)
                            elif part == "references":
                                stats_key_to_reference_ids[stats_key].add(id)
    print("done")
    
    return stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts

SyftSuccess: Syft function 'compute_document_data_overlap' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [11]:
client.code.submit(compute_document_data_overlap)

SyftSuccess: User Code Submitted

In [12]:
@sy.syft_function_single_use(input_files=input_files_ptr, scenario_files=scenario_files_ptr)
def main_function(domain, input_files, scenario_files):
    N = [5, 9, 13]
    jobs = []
    for n in N[:1]:
        for scenario_file in scenario_files:
            batch_job = domain.launch_job(
                compute_document_data_overlap,
                scenario_file=scenario_file,
                input_files=input_files,
                n=n
            )
            jobs.append(batch_job)

    return None


SyftSuccess: Syft function 'main_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [13]:
client.code.request_code_execution(main_function)
client.requests[-1].approve()

Request approved for domain test-domain-helm2


SyftSuccess: Request 8a84551794dc49cda40f4d683fb19bdf changes applied

In [14]:
job = client.code.main_function(input_files=input_files_ptr, scenario_files=scenario_files_ptr, blocking=False)

# Get results

In [15]:
job

```python
class Job:
    id: UID = 585159e4b6dd4599a3f0db6755e676f3
    status: created
    has_parent: False
    result: None
    logs:

0 
    
```

In [16]:
job.wait().get()

LAUNCHING JOB compute_document_data_overlap
LAUNCHING JOB compute_document_data_overlap


FUNCTION LOG (80211968cab84fc0bbc0b6d7d986fa2f): starting overlap computation
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): starting overlap computation
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): preparing scenarios and creating indexes
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): computing overlap
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): preparing scenarios and creating indexes
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): computing overlap
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): done
FUNCTION LOG (ba3acba2f0b74644a1c61565187d21b4): done


In [17]:
job.subjobs

[syft.service.job.job_stash.Job, syft.service.job.job_stash.Job]

In [18]:
job.subjobs[0].logs()

starting overlap computation
preparing scenarios and creating indexes
computing overlap
preparing scenarios and creating indexes
done
computing overlap
done



In [19]:
results = [j.wait().get() for j in job.subjobs]

In [20]:
#stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts
results

[(defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135})),
 (defaultdict(set,
              {'philosophy_test_5': {'id328'},
               'philosophy_valid_5': {'id12'}}),
  defaultdict(set, {}),
  defaultdict(int,
              {'philosophy_train_5': 5,
               'philosophy_valid_5': 34,
               'philosophy_test_5': 311,
               'anatomy_train_5': 5,
               'anatomy_valid_5': 14,
               'anatomy_test_5': 135}))]

In [21]:
results[0]

(defaultdict(set,
             {'philosophy_test_5': {'id328'}, 'philosophy_valid_5': {'id12'}}),
 defaultdict(set, {}),
 defaultdict(int,
             {'philosophy_train_5': 5,
              'philosophy_valid_5': 34,
              'philosophy_test_5': 311,
              'anatomy_train_5': 5,
              'anatomy_valid_5': 14,
              'anatomy_test_5': 135}))

# Aggregate

In [22]:
stats_key_to_input_ids, stats_key_to_reference_ids, stats_key_counts = zip(*results)

total_input_ids = defaultdict(set)
total_reference_ids = defaultdict(set)
total_stats_key_counts = defaultdict(int)

for d in stats_key_counts:
    for key, val in d.items():
        total_stats_key_counts[key] += val


for d in stats_key_to_input_ids:
    for key in d:
        new_set = set()
        if key in total_input_ids:
            new_set = total_input_ids[key]
        new_set = new_set.union(d[key])
        total_input_ids[key] = new_set

for d in stats_key_to_reference_ids:
    for key in d:
        new_set = set()
        if key in total_reference_ids:
            new_set = total_reference_ids[key]
        new_set = total_reference_ids[key].union(d[key])
        total_reference_ids[key] = new_set

all_data_overlap_stats = []
for stats_key, count in total_stats_key_counts.items():
    data_overlap_stats = {
        'data_overlap_stats_key': None,
        'num_instances': count,
        'instance_ids_with_overlapping_input': sorted(total_input_ids[stats_key]),
        'instance_ids_with_overlapping_reference': sorted(total_reference_ids[stats_key]),
    }
    subject, split, n_str = stats_key.split('_')
    data_overlap_stats['data_overlap_stats_key'] = {
        'light_scenario_key': {'subject': subject, 'split': split},
        'overlap_protocol_spec': {'n': int(n_str)}
    }
    all_data_overlap_stats.append(data_overlap_stats)




In [23]:
from pprint import pprint
pprint(all_data_overlap_stats)

[{'data_overlap_stats_key': {'light_scenario_key': {'split': 'train',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': [],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 10},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'valid',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id12'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 68},
 {'data_overlap_stats_key': {'light_scenario_key': {'split': 'test',
                                                    'subject': 'philosophy'},
                             'overlap_protocol_spec': {'n': 5}},
  'instance_ids_with_overlapping_input': ['id328'],
  'instance_ids_with_overlapping_reference': [],
  'num_instances': 